In [36]:
import requests
from bs4 import BeautifulSoup

In [37]:
url='https://www.imdb.com/chart/top/'

In [43]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
response=requests.get(url,headers=headers)
content=response.content

In [44]:
soup=BeautifulSoup(content,'html.parser')

In [59]:
#hyper_l=soup.find_all('class':'ipc-title-link-wrapper','a')
hyper_l=soup.find_all('a', class_='ipc-title-link-wrapper')

In [68]:
href_list=[a.get('href') for a in hyper_l if str(a.get('href')).split('/')[1]!='chart' ] 

In [181]:
i=0
for q in range(0,9):
    for w in range(0,9):
        for e in range(0,9):
            for r in range(0,9):
                for t in range(0,9):
                    for y in range(0,9):
                        for u in range(4,9):
                            w_url='https://www.imdb.com/title/tt'+str(q)+str(w)+str(e)+str(r)+str(t)+str(y)+str(u)+'/'
                            response=requests.get(w_url,headers=headers)
                            content=response.content
                            soup=BeautifulSoup(content,'html.parser')
                            title=soup.find('span',class_='sc-afe43def-1 fDTGTb').text
                            try:
                             storyline=soup.find_all('div',class_='ipc-html-content-inner-div')[2].text
                            except:
                                storyline='NaN'

                            Director=soup.find_all('a',class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')[0].text
                            i=i+1
                            print(i,w_url)
                        
                            break
                        break                   
                    break
                break
            break
        break
    break


                        

1 https://www.imdb.com/title/tt0000004/


In [216]:
Date=soup.find_all('ul',class_='ipc-metadata-list ipc-metadata-list--dividers-all ipc-metadata-list--base')
Date=(Date[0].text.strip('Release date')).split(' ')
print(Date[0:3])

['October', '28,', '1892']


In [213]:
for d in Date.split(' '):
    print(d)

October
28,
1892
(France)Country
of
originFranceLanguageNoneAlso
known
asПолная
кружка
пиваSee
more
company
credits
at
IMDbPro


In [156]:
soup.find_all('div',class_='ipc-overflowText ipc-overflowText--pageSection ipc-overflowText--base') #ipc-html-content-inner-div

[]

In [139]:
Date=soup.find_all('a',class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
print(Date)

[<a aria-disabled="false" class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link" href="/name/nm0005690/?ref_=tt_ov_dr" role="button" tabindex="0">William K.L. Dickson</a>, <a aria-disabled="false" class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link" href="/name/nm1588970/?ref_=tt_ov_st" role="button" tabindex="0">Carmencita</a>, <a aria-disabled="false" class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link" href="/name/nm0005690/?ref_=tt_ov_dr" role="button" tabindex="0">William K.L. Dickson</a>, <a aria-disabled="false" class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link" href="/name/nm1588970/?ref_=tt_ov_st" role="button" tabindex="0">Carmencita</a>, <a aria-disabled="false" class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link" href="/name/nm0005690/?ref_=tt_cl_dr_1" 

In [141]:
print(len(Date))
for g in Date:
    print(g.text)

12
William K.L. Dickson
Carmencita
William K.L. Dickson
Carmencita
William K.L. Dickson
March 10, 1894 (United States)
United States
None
West Orange, New Jersey, USA
Edison Manufacturing Company
Black and White
Silent
